In [2]:
import numpy as np
from mealpy.optimizer import Optimizer
import pandas as pd
import os

optimizer = Optimizer()

ModuleNotFoundError: No module named 'numpy'

# Add csv files to BSE_30 folder and convert open price of each csv to a df

In [2]:
company_list = []
data = {}

In [3]:
folder_path = 'BSE_30'

files = os.listdir(folder_path)

for file_name in files:
    file_path = os.path.join(folder_path, file_name)
    file_name = file_path[7:-11]
    file = pd.read_csv(file_path)
    data[file_name] = file['Open Price'].tolist() 

In [4]:
df = pd.DataFrame(data)
axis = df.shape

In [5]:
# df=pd.read_csv("500112.csv")
# axis =df.shape

# Reading usefull information form the df creating eg. Mean, Returns, geomean, sd

In [6]:
no_of_stocks = axis[1]
columns = df.columns

In [7]:
# Step 1 calculate returns (stockT2 - stockT1)/stockT1
returns = pd.DataFrame(index=range(axis[0]), columns=range(axis[1]))
for idx, col in enumerate(columns):
    for i in range(1, len(df[col])):
        s1 = df[col][i - 1]
        s2 = df[col][i]
        return_value = (s2 - s1) / s1
        returns[idx][i] = return_value

# returns = returns*100
returns.drop([0], axis=0, inplace=True)

In [8]:
returns

,0,1,2
1,-0.005486,-0.034218,-0.014587
2,0.00495,0.01492,-0.012476
3,0.007927,0.000607,0.008651
4,-0.018675,0.025979,-0.007875
5,-0.004359,-0.037005,-0.006892
6,-0.003014,-0.006562,0.000015
7,0.003986,0.002471,0.008326
8,-0.013022,-0.01351,-0.008655
9,0.026166,0.05228,0.01825
10,-0.01922,-0.035618,-0.012013


In [9]:
# Step 2 Calculation of covraince matrix and mean
Σ = returns.cov()
µ = returns.mean()

In [10]:
# Step 4 calcualting geomean of returns constraint 1
returns_for_geomean = returns + 1
returns_for_geomean[0]
geo_mean = [
    (returns_for_geomean[i].prod() ** (1 / len(returns_for_geomean[i])) - 1)
    for i in returns_for_geomean
]
geo_mean = pd.DataFrame(geo_mean)
geo_mean

,0
0,0.003217
1,-0.000024
2,0.001979


In [11]:
lowerbound = [0 for i in range(geo_mean.shape[0])]
upperbound = [1 for i in range(geo_mean.shape[0])]

# This will tell whether we are solving for mini or not.
mini = True
T = 100
N = 1000
# optimum_given = float("inf")
Dim = geo_mean.shape[0]
Dim

3

In [12]:
def franctional_constraints(a, b, tpe, value):
    if b == 0:
        return False
    if tpe == "le":
        return (a / b) > value
    if tpe == "ge":
        return (a / b) < value
    if tpe == "g":
        return (a / b) <= value
    if tpe == "l":
        return (a / b) >= value

In [13]:
def not_in_range(lowerbound, upperbound, x):
    geo_x = pd.DataFrame(x)
    c_1_n = np.dot(geo_mean.T, geo_x)[0]
    c_1_d = 1
    c_2_n = np.sum(x)
    c_2_d = 1
    c_3_n = np.sum(x)
    c_3_d = 1
    if (
        franctional_constraints(c_1_n, c_1_d, "ge", 0.001)
        or franctional_constraints(c_2_n, c_2_d, "ge", 0.99)
        or franctional_constraints(c_3_n, c_3_d, "le", 1.01)
    ):
        return True

    for i in range(0, len(lowerbound)):
        if not lowerbound[i] <= x[i] <= upperbound[i]:
            return True

    return False

In [14]:
Σ

,0,1,2
0,0.000124,0.000098,0.000025
1,0.000098,0.000619,0.000143
2,0.000025,0.000143,0.000234


In [15]:
def fitness_function(x):
    xT = np.transpose(x)
    # geo_x = pd.DataFrame(x)
    a = np.dot(np.dot(xT, Σ), x)
    b = 1
    if b == 0:
        return False

    return -a / b if mini else a / b

In [16]:
population = np.transpose(
    [np.random.uniform(lowerbound[i], upperbound[i], N) for i in range(Dim)]
)

In [17]:
t = 0
while t != T:
    # Initialize the best rabbit location (global best)
    best_rabbit_location = None
    best_rabbit_fitness = float("inf")
    fitness_values = [fitness_function(hawks) for hawks in population]
    best_hawk_index = np.argmax(fitness_values)

    if fitness_values[best_hawk_index] < best_rabbit_fitness:
        best_rabbit_location = population[best_hawk_index].copy()
        best_rabbit_fitness = fitness_values[best_hawk_index]

    population_new = []

    i = 0
    while i != N:
        E0 = 2 * np.random.uniform() - 1
        J0 = 2 * (1 - np.random.uniform())
        E = 2 * E0 * (1 - (t + 1) * 1.0 / T)

        # exploration phase
        if np.abs(E) >= 1:
            if np.random.rand() >= 0.5:  # perch based on other family members
                X_rand = population[np.random.randint(0, N)]

                pos_new = X_rand - np.random.uniform() * np.abs(
                    X_rand - 2 * np.random.uniform() * population[i]
                )

            else:
                x_mean = np.mean([x for x in np.transpose(population)])
                pos_new = (best_rabbit_location - x_mean) - np.random.uniform() * (
                    np.array(lowerbound)
                    + np.random.uniform()
                    * (np.array(upperbound) - np.array(lowerbound))
                )

            if not_in_range(lowerbound, upperbound, pos_new):
                continue
            population_new.append(pos_new)
        else:
            if np.random.rand() >= 0.5:
                if np.abs(E) >= 0.5:  # Soft besiege Eq. (6) in paper
                    pos_new = (
                        best_rabbit_location
                        - population[i]
                        - E * np.abs(J0 * best_rabbit_location - population[i])
                    )
                else:  # Hard besiege Eq. (4) in paper
                    pos_new = best_rabbit_location - E * np.abs(
                        best_rabbit_location - population[i]
                    )

                if not_in_range(lowerbound, upperbound, pos_new):
                    continue
                population_new.append(pos_new)

            else:
                LF_D = optimizer.get_levy_flight_step(
                    beta=1.5, multiplier=0.01, case=-1
                )
                if np.abs(E) >= 0.5:
                    Y = best_rabbit_location - E * np.abs(
                        J0 * best_rabbit_location - population[i]
                    )
                else:
                    x_mean = np.mean(population)
                    Y = best_rabbit_location - E * np.abs(
                        J0 * best_rabbit_location - x_mean
                    )

                S = np.random.uniform(lowerbound, upperbound)
                Z = Y + S * LF_D

                fitness_of_Y = fitness_function(Y)
                fitness_of_Z = fitness_function(Z)
                if not fitness_of_Y or not fitness_of_Z:
                    continue

                if fitness_function(Y) < fitness_values[i]:
                    pos_new = Y
                elif fitness_function(Z) < fitness_values[i]:
                    pos_new = Z
                else:
                    pos_new = population[i]

                if not_in_range(lowerbound, upperbound, pos_new):
                    continue

                population_new.append(pos_new)
        i += 1
    population = population_new
    # if np.abs(best_rabbit_fitness - optimum_given) < 0.001:
    #     break
    print(t)
    t += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [18]:
formatted_best_rabbit_location = []
for e in best_rabbit_location:
    formatted_result = "{:.3f}".format(e)
    formatted_best_rabbit_location.append(formatted_result)

best_rabbit_fitness = best_rabbit_fitness if not mini else -best_rabbit_fitness
formatted_best_rabbit_fitness = "{:.15f}".format(best_rabbit_fitness)
print("*********************************************************************")
print("best rabbit location = " + str(formatted_best_rabbit_location))
print("best rabbit fitness = " + str(formatted_best_rabbit_fitness))

*********************************************************************
best rabbit location = ['0.699', '0.000', '0.290']
best rabbit fitness = 0.000090581947454


In [21]:
# rf = 0.072
xTΣx = np.dot(np.dot(best_rabbit_location, Σ), np.transpose(best_rabbit_location))
print(xTΣx)
µTx = np.dot(µ.T, best_rabbit_location)
numerator = µTx - rf
denominator = np.sqrt(xTΣx)
sparse_ratio = numerator / denominator
print(sparse_ratio)

9.058194745412231e-05
-7.260363012301454
